In [1]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
#data_path = "\\users\\user\\onedrive\\a7\\Data"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 6, 27)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-06-27
yesterday: 2023-06-26


#### Restart and Run All

In [3]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [4]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1272796,TTB,2023-06-26,2023-06-26,0,1.58,1.60,0.02,1.27%,2,0,1.64,1.55,"3,314,542,691",2023-06-26,2023-06-26,1


In [5]:
names = tmp["name"]
type(names)

pandas.core.series.Series

In [6]:
name = names.to_string(index=False)
type(name)

str

In [7]:
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'TTB'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9717,TTB,0.00,0.00,O,1.09,0.00,0.00,0.00,0.00,"20,000",-9,17,0,0,0,52WL,SET50


### End of Tables in the process

In [8]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2023-06-26'
ORDER BY a.name



,name
0,AH
1,AWC
2,BANPU
3,BBL
4,BCH
5,BDMS
6,BEM
7,BH
8,CPNREIT
9,DIF


In [9]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-06-26' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [10]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,BANPU,2023-06-26,2023-06-26,9.15,9.00,"109,755,280",9.15,9.00,B,SET50,-0.15,-1.64%
1,CPNREIT,2023-06-26,2023-06-26,11.00,10.70,"10,290,298",11.00,10.20,B,SET,-0.30,-2.73%
2,DIF,2023-06-09,2023-06-26,12.30,10.60,"832,864,684",12.40,10.00,B,SET,-1.70,-13.82%
3,JMART,2023-06-15,2023-06-26,19.30,13.10,"340,431,514",20.10,13.10,B,SET50,-6.20,-32.12%
4,JMT,2023-06-21,2023-06-26,38.75,33.50,"93,559,290",41.00,33.50,B,SET50,-5.25,-13.55%
5,MCS,2023-06-14,2023-06-26,7.10,6.65,"8,899,812",7.15,6.55,B,SET,-0.45,-6.34%
6,SINGER,2023-06-23,2023-06-26,9.85,8.25,"10,493,538",10.10,8.25,B,SET100,-1.60,-16.24%
7,STA,2023-06-19,2023-06-26,18.80,17.00,"84,246,056",19.30,17.00,B,SET100,-1.80,-9.57%
8,SYNEX,2023-06-20,2023-06-26,12.80,12.10,"1,259,506",13.10,12.10,B,SET,-0.70,-5.47%
9,TMT,2023-06-21,2023-06-26,7.45,7.25,"1,334,174",7.50,7.25,B,SET,-0.20,-2.68%


In [11]:
df.shape

(35, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [12]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,BANPU,2023-06-26,2023-06-26,9.15,9.00,"109,755,280",9.15,9.00,-1.64%,B
1,CPNREIT,2023-06-26,2023-06-26,11.00,10.70,"10,290,298",11.00,10.20,-2.73%,B
2,DIF,2023-06-09,2023-06-26,12.30,10.60,"832,864,684",12.40,10.00,-13.82%,B
3,JMART,2023-06-15,2023-06-26,19.30,13.10,"340,431,514",20.10,13.10,-32.12%,B
4,JMT,2023-06-21,2023-06-26,38.75,33.50,"93,559,290",41.00,33.50,-13.55%,B
5,MCS,2023-06-14,2023-06-26,7.10,6.65,"8,899,812",7.15,6.55,-6.34%,B
6,SINGER,2023-06-23,2023-06-26,9.85,8.25,"10,493,538",10.10,8.25,-16.24%,B
7,STA,2023-06-19,2023-06-26,18.80,17.00,"84,246,056",19.30,17.00,-9.57%,B
8,SYNEX,2023-06-20,2023-06-26,12.80,12.10,"1,259,506",13.10,12.10,-5.47%,B
9,TMT,2023-06-21,2023-06-26,7.45,7.25,"1,334,174",7.50,7.25,-2.68%,B


In [13]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [14]:
df[cols].shape

(35, 10)

In [15]:
sales = df[cols]
sales

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,BANPU,2023-06-26,2023-06-26,9.15,9.00,109755280,9.15,9.00,-1.64,B
1,CPNREIT,2023-06-26,2023-06-26,11.00,10.70,10290298,11.00,10.20,-2.73,B
2,DIF,2023-06-09,2023-06-26,12.30,10.60,832864684,12.40,10.00,-13.82,B
3,JMART,2023-06-15,2023-06-26,19.30,13.10,340431514,20.10,13.10,-32.12,B
4,JMT,2023-06-21,2023-06-26,38.75,33.50,93559290,41.00,33.50,-13.55,B
5,MCS,2023-06-14,2023-06-26,7.10,6.65,8899812,7.15,6.55,-6.34,B
6,SINGER,2023-06-23,2023-06-26,9.85,8.25,10493538,10.10,8.25,-16.24,B
7,STA,2023-06-19,2023-06-26,18.80,17.00,84246056,19.30,17.00,-9.57,B
8,SYNEX,2023-06-20,2023-06-26,12.80,12.10,1259506,13.10,12.10,-5.47,B
9,TMT,2023-06-21,2023-06-26,7.45,7.25,1334174,7.50,7.25,-2.68,B


In [16]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
prices = pd.read_csv(input_file)
prices

,name,date,price,change,%change,open,high,low,volume,value(m)
0,ACE,2023-06-27,1.88,0.01,0.534759,1.90,1.91,1.87,11066943,20939.21488
1,ADVANC,2023-06-27,210.00,0.00,0.000000,211.00,213.00,210.00,3277070,692161.03700
2,AEONTS,2023-06-27,185.00,-2.50,-1.333333,187.00,187.00,185.00,86402,16051.17400
3,AH,2023-06-27,32.25,1.75,5.737705,30.75,32.50,30.75,3038977,96992.89475
4,AIE,2023-06-27,1.30,-0.06,-4.411765,1.32,1.38,1.30,815426,1079.18532
...,...,...,...,...,...,...,...,...,...,...
187,WHAIR,2023-06-27,7.15,-0.05,-0.694444,7.15,7.15,7.10,521938,3716.68125
188,WHART,2023-06-27,10.40,-0.20,-1.886792,10.60,10.60,10.40,1488310,15621.98530
189,WHAUP,2023-06-27,3.64,-0.04,-1.086957,3.70,3.72,3.64,1763485,6473.06440
190,WICE,2023-06-27,8.20,0.05,0.613497,8.15,8.35,8.10,827660,6816.61300


In [17]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,date,price,change,%change,open,high,low,volume,value(m)
0,BANPU,2023-06-26,2023-06-26,9.15,9.00,109755280,9.15,9.00,-1.64,B,2023-06-27,8.90,-0.10,-1.111111,9.05,9.05,8.90,47782718,428901.01290
1,CPNREIT,2023-06-26,2023-06-26,11.00,10.70,10290298,11.00,10.20,-2.73,B,2023-06-27,11.40,0.70,6.542056,11.30,11.90,11.20,12987769,149252.23470
2,DIF,2023-06-09,2023-06-26,12.30,10.60,832864684,12.40,10.00,-13.82,B,2023-06-27,10.70,0.10,0.943396,10.60,10.80,10.60,27576708,295398.26790
3,JMART,2023-06-15,2023-06-26,19.30,13.10,340431514,20.10,13.10,-32.12,B,2023-06-27,13.70,0.60,4.580153,13.40,14.40,13.10,52328753,721472.08770
4,JMT,2023-06-21,2023-06-26,38.75,33.50,93559290,41.00,33.50,-13.55,B,2023-06-27,34.75,1.25,3.731343,33.50,35.50,33.50,18436231,643381.04375
5,MCS,2023-06-14,2023-06-26,7.10,6.65,8899812,7.15,6.55,-6.34,B,2023-06-27,6.45,-0.20,-3.007519,6.60,6.65,6.45,848822,5529.12035
6,SINGER,2023-06-23,2023-06-26,9.85,8.25,10493538,10.10,8.25,-16.24,B,2023-06-27,8.70,0.45,5.454545,8.35,8.85,8.00,16898481,144335.19645
7,STA,2023-06-19,2023-06-26,18.80,17.00,84246056,19.30,17.00,-9.57,B,2023-06-27,16.80,-0.20,-1.176471,17.10,17.20,16.70,3881817,65687.07970
8,SYNEX,2023-06-20,2023-06-26,12.80,12.10,1259506,13.10,12.10,-5.47,B,2023-06-27,12.30,0.20,1.652893,12.20,12.40,11.90,197848,2417.37840
9,TMT,2023-06-21,2023-06-26,7.45,7.25,1334174,7.50,7.25,-2.68,B,2023-06-27,7.30,0.05,0.689655,7.25,7.30,7.20,36725,266.48175


In [18]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
colu

['name',
 'fm_date',
 'to_date',
 'fm_price',
 'to_price',
 'qty',
 'max_price',
 'min_price',
 'percent',
 'status',
 'price',
 'change',
 'volume',
 'date']

In [19]:
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,BANPU,2023-06-26,2023-06-26,9.15,9.00,109755280,9.15,9.00,-1.64,B,8.90,-0.10,47782718,2023-06-27
1,CPNREIT,2023-06-26,2023-06-26,11.00,10.70,10290298,11.00,10.20,-2.73,B,11.40,0.70,12987769,2023-06-27
2,DIF,2023-06-09,2023-06-26,12.30,10.60,832864684,12.40,10.00,-13.82,B,10.70,0.10,27576708,2023-06-27
3,JMART,2023-06-15,2023-06-26,19.30,13.10,340431514,20.10,13.10,-32.12,B,13.70,0.60,52328753,2023-06-27
4,JMT,2023-06-21,2023-06-26,38.75,33.50,93559290,41.00,33.50,-13.55,B,34.75,1.25,18436231,2023-06-27
5,MCS,2023-06-14,2023-06-26,7.10,6.65,8899812,7.15,6.55,-6.34,B,6.45,-0.20,848822,2023-06-27
6,SINGER,2023-06-23,2023-06-26,9.85,8.25,10493538,10.10,8.25,-16.24,B,8.70,0.45,16898481,2023-06-27
7,STA,2023-06-19,2023-06-26,18.80,17.00,84246056,19.30,17.00,-9.57,B,16.80,-0.20,3881817,2023-06-27
8,SYNEX,2023-06-20,2023-06-26,12.80,12.10,1259506,13.10,12.10,-5.47,B,12.30,0.20,197848,2023-06-27
9,TMT,2023-06-21,2023-06-26,7.45,7.25,1334174,7.50,7.25,-2.68,B,7.30,0.05,36725,2023-06-27


In [20]:
df_merge[colu].shape

(35, 14)

In [21]:
df_merge.shape

(35, 19)

In [22]:
file_name = "daily-sales-prices.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(data_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(one_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

In [23]:
### Call ruby ruby\daily-out-new.rb

In [24]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [25]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [26]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AH       2023-06-27 2023-06-27  30.50  32.25   5.74%   4,261,614  33.00  30.50 I Insert
AWC      2023-06-21 2023-06-27   4.90   4.76  -2.86% 173,870,206   5.05   4.76 I Update
BANPU    2023-06-26 2023-06-27   9.15   8.90  -2.73% 109,755,280   9.15   8.90 B Update
BBL      2023-06-27 2023-06-27 163.00 161.50  -0.92%  36,467,586 164.00 160.50 O Insert
BCH      2023-06-23 2023-06-27  17.60  17.00  -3.41%  18,631,372  17.70  17.00 I Update
BDMS     2023-06-27 2023-06-27  28.50  28.00  -1.75%  78,975,939  28.75  28.00 O Insert
BEM      2023-06-21 2023-06-27   8.40   8.20  -2.38% 152,460,902   8.50   8.10 O Update
BH       2023-06-23 2023-06-27 228.00 225.00  -1.32%   3,076,606 231.00 225.00 O Update
CPNREIT  2023-06-27 2023-06-27  10.70  11.40   6.54%  10,290,298  11.40  10.20 B Insert
DIF      2023-06-27 2023-06-27  

In [27]:
os.chdir("C:\\users\\user\\onedrive\\a7\\Daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\Daily'

### cd\ruby\port_lite
### rails runner db/ins_sales.rb